<a href="https://colab.research.google.com/github/NatsuiroGinga/pycorrector/blob/master/pycorrector_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文本纠错

## 1 下载与导入

In [12]:
!pip install pandas transformers datasets peft accelerate -U

In [4]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [5]:
import sys
import gradio as gr
import torch
from transformers import BertTokenizerFast, BertForMaskedLM

## 2 Gradio

In [6]:
def predict(text):
    return model.correct(text)


if __name__ == '__main__':
    model = MacBertCorrector()

    examples = [
        ['真麻烦你了。希望你们好好的跳无'],
        ['少先队员因该为老人让坐'],
        ['机七学习是人工智能领遇最能体现智能的一个分知'],
        ['今天心情很好'],
        ['他法语说的很好，的语也不错'],
        ['他们的吵翻很不错，再说他们做的咖喱鸡也好吃'],
    ]

    gr.Interface(
        predict,
        inputs="text",
        outputs="text",
        title="Chinese Spelling Correction Model",
        description="Copy or input error Chinese text. Submit and the machine will correct text.",
        article="Link to github: <a href='https://github.com/shibing624/pycorrector' style='color:blue;' target='_blank\'>pycorrector</a>",
        examples=examples
    ).launch()

NameError: name 'MacBertCorrector' is not defined

## 3 MacBert-CSC

In [10]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizerFast.from_pretrained("shibing624/macbert4csc-base-chinese")
model = BertForMaskedLM.from_pretrained("shibing624/macbert4csc-base-chinese")
model.to(device)

texts = ["C请介绍一下", "你找到你最喜欢的工作，我也很高心。"]
with torch.no_grad():
    outputs = model(**tokenizer(texts, padding=True, return_tensors='pt').to(device))

result = []
for ids, text in zip(outputs.logits, texts):
    _text = tokenizer.decode(torch.argmax(ids, dim=-1), skip_special_tokens=True).replace(' ', '')
    corrected_text = _text[:len(text)]
    print(text, ' => ', corrected_text)
    result.append(corrected_text)
print(result)

C请介绍一下  =>  c请介绍一下
你找到你最喜欢的工作，我也很高心。  =>  你找到你最喜欢的工作，我也很高兴。
['c请介绍一下', '你找到你最喜欢的工作，我也很高兴。']


## 4 GPT/Qwen

In [17]:
prompt = """
# 系统角色指令： 你是一名专业的中文文本纠错模型，能够识别并纠正文本中的错别字、多字、漏字，以及常见词汇和领域术语的误用。

# 对模型的主要要求：

1. 请专注于纠正输入文本中的语言错误，尽量保持原意。

2. 当出现领域专有名词（如公司名称、产品名称、技术术语等）时，请参照以下上下文信息进行纠正。

3. 对于没有明显错误或不影响理解的部分，不要进行改动。

4. 仅输出纠正后的文本，不要输出任何额外解释或标记。

# 上下文信息（示例）：

1. 真视通：公司名，与“真通智用”“整通只用” 等易混淆

2. 紫荆视通：公司名，与“紫金系统”等相近但不相同

3. 双相液冷板：正确技术名词，与“双像叶冷板”“双向叶冷板” 相混淆

"""

print(prompt)


# 系统角色指令： 你是一名专业的中文文本纠错模型，能够识别并纠正文本中的错别字、多字、漏字，以及常见词汇和领域术语的误用。

# 对模型的主要要求：

1. 请专注于纠正输入文本中的语言错误，尽量保持原意。

2. 当出现领域专有名词（如公司名称、产品名称、技术术语等）时，请参照以下上下文信息进行纠正。

3. 对于没有明显错误或不影响理解的部分，不要进行改动。

4. 仅输出纠正后的文本，不要输出任何额外解释或标记。

# 上下文信息（示例）：

1. 真视通：公司名，与“真通智用”“整通只用” 等易混淆

2. 紫荆视通：公司名，与“紫金系统”等相近但不相同

3. 双相液冷板：正确技术名词，与“双像叶冷板”“双向叶冷板” 相混淆


